# <strong>Bitcoin Recurrent Neural Network<strong>
### Justin Marlor & Habit Blunk
##### *Colorado State University*

In [3]:
import pandas as pd
import subprocess

result = subprocess.run(['bash', './add-run-kaggle-bitcoin'], capture_output=True,text=True)

print(result.stdout)
print(result.stderr)

if result.returncode == 0:
  df = pd.read_csv("kaggle-bitcoin/upload/btcusd_1-min_data.csv", dtype={"Volume": float}, low_memory=False)
  display(df)

Installing dependencies from lock file

No dependencies to install or update

Installing the current project: kaggle-bitcoin (0.1.0)
Current time (UTC): 2025-04-08 18:40:17.300991+00:00
Dataset URL: https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data
Checking for missing data...
Last data point in dataset: 2025-03-17 00:00:00+00:00 (Unix: 1742169600)
Current time (minus buffer): 2025-04-08 18:33:38.538365+00:00 (Unix: 1744137218)
Gap of 1967618 seconds detected.
Missing data detected.
Fetching data from 2025-03-17 00:00:00+00:00 to 2025-04-08 18:33:38+00:00
Fetching chunk 1/33: 2025-03-17 00:00:00+00:00 to 2025-03-17 16:40:00+00:00
  - Retrieved 1000 data points
Fetching chunk 2/33: 2025-03-17 16:40:00+00:00 to 2025-03-18 09:20:00+00:00
  - Retrieved 1000 data points
Fetching chunk 3/33: 2025-03-18 09:20:00+00:00 to 2025-03-19 02:00:00+00:00
  - Retrieved 1000 data points
Fetching chunk 4/33: 2025-03-19 02:00:00+00:00 to 2025-03-19 18:40:00+00:00
  - Retrieved 1000 data

,Timestamp,Open,High,Low,Close,Volume,datetime
0,1.325412e+09,4.58,4.58,4.58,4.58,0.000000,2012-01-01 10:01:00+00:00
1,1.325412e+09,4.58,4.58,4.58,4.58,0.000000,2012-01-01 10:02:00+00:00
2,1.325412e+09,4.58,4.58,4.58,4.58,0.000000,2012-01-01 10:03:00+00:00
3,1.325412e+09,4.58,4.58,4.58,4.58,0.000000,2012-01-01 10:04:00+00:00
4,1.325412e+09,4.58,4.58,4.58,4.58,0.000000,2012-01-01 10:05:00+00:00
...,...,...,...,...,...,...,...
6977588,1.744137e+09,77317.00,77317.00,77271.00,77271.00,0.045864,NaN
6977589,1.744137e+09,77262.00,77932.00,77262.00,77755.00,1.504965,NaN
6977590,1.744137e+09,77777.00,77777.00,77525.00,77579.00,3.720058,NaN
6977591,1.744137e+09,77644.00,77708.00,77592.00,77592.00,0.056099,NaN


In [5]:
df['datetime'] = pd.to_datetime(df['Timestamp'].astype('Int64'), unit='s', errors='coerce')
df['Year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
print(df)

            Timestamp      Open      High       Low     Close    Volume  \
0        1.325412e+09      4.58      4.58      4.58      4.58  0.000000   
1        1.325412e+09      4.58      4.58      4.58      4.58  0.000000   
2        1.325412e+09      4.58      4.58      4.58      4.58  0.000000   
3        1.325412e+09      4.58      4.58      4.58      4.58  0.000000   
4        1.325412e+09      4.58      4.58      4.58      4.58  0.000000   
...               ...       ...       ...       ...       ...       ...   
6976229  1.744055e+09  78230.00  78230.00  78052.00  78054.00  2.234757   
6976230  1.744055e+09  78059.00  78214.00  78047.00  78188.00  3.388673   
6976231  1.744056e+09  78116.00  78217.00  78116.00  78202.00  1.383640   
6976232  1.744056e+09  78209.00  78258.00  78080.00  78080.00  3.512614   
6976233  1.744056e+09  78073.00  78278.00  78070.00  78262.00  1.907790   

                   datetime  Year  month  day  
0       2012-01-01 10:01:00  2012      1    1  
1  

In [4]:
import torch
import torch.nn as nn
from torch.nn import GRU
from torch.nn import RNN
import torch.utils.data as data_utils
input_size = df.shape[1]
tensor = data_utils.DataLoader(df)

# Input size: features presented to model
# Hidden size: number of connections made by each layer of the NN, We will tune this using a logarithmic scale
# num_layers: number of trained RNNs being stacked on top of eachother (learn from eachother).
rnn = nn.RNN(input_size=input_size, hidden_size=n, num_layers=2, batch_first=True)

NameError: name 'n' is not defined